# 간단한 전이학습 시켜보기

1. 주제를 정한다.
2. 주제에 맞는 태스크를 판단한다.
3. 해당 태스크를 수행하는 모델을 huggingface에서 찾는다.
4. 적절한 데이터셋을 정제하여 준비한다.
5. 3에서 찾은 모델을 로드하여 '간단하게' 전이학습을 진행한다.
6. 추론 결과를 확인한다.

### 데이터 출처: https://www.kaggle.com/datasets/datasnaek/mbti-type
### 모델 출처: https://huggingface.co/mradermacher/English-to-korean-translator-GGUF

In [57]:
import numpy as np
import pandas as pd

mbti_df = pd.read_csv('twitter_MBTI.csv')
print(mbti_df.info())
print(mbti_df['text'].value_counts())
mbti_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7811 entries, 0 to 7810
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  7811 non-null   int64 
 1   text        7811 non-null   object
 2   label       7811 non-null   object
dtypes: int64(1), object(2)
memory usage: 183.2+ KB
None
text
I just realize that today is dzi's birthday... It feels so bittersweet bc I had a thing for a certain redhead varia… https://t.co/zk4pCuOW4Z|||我應不應該又玩nucarni? 我想練習讀中文。|||@Sakunyuw ... Je suis désolée.|||@Sakunyuw Ce qui va se passer?|||@curemachwindy Is it available online or in theaters? I don't want to jump to conclusions too quickly. However, I g… https://t.co/AcJdHogVyt|||The one college major I keep coming back to especially when I am overwhelmed and upset matters more than mb/ti stuf… https://t.co/pW7930Zh5k|||I wish I can say the same about Mandarin language films but I probably need more eng subs.|||The feeling when 

,Unnamed: 0,text,label
0,0,@Pericles216 @HierBeforeTheAC @Sachinettiyil T...,intj
1,1,@Hispanthicckk Being you makes you look cute||...,intj
2,2,@Alshymi Les balles sont réelles et sont tirée...,intj
3,3,"I'm like entp but idiotic|||Hey boy, do you wa...",intj
4,4,@kaeshurr1 Give it to @ZargarShanif ... He has...,intj
...,...,...,...
7806,7806,"@sobsjjun God,,pls take care 😕|||@sobsjjun Hir...",intp
7807,7807,@Ignis_02 wow last time i got intp https://t.c...,intp
7808,7808,@akupilled A 100%|||@akupilled That SOMEONE wi...,entp
7809,7809,If you’re #INTJ this one is for you | What is ...,infj


In [19]:
from transformers import pipeline, AutoTokenizer

MODEL = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

translator = pipeline(
    "translation",
    model=MODEL,
    tokenizer=tokenizer,
    src_lang="eng_Latn",   # 영어 BCP-47
    tgt_lang="kor_Hang",   # 한국어 BCP-47
    max_length=1024
)

print(translator("We propose a simple yet effective approach.")[0]["translation_text"])

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

Device set to use mps:0


우리는 간단하면서도 효과적인 접근법을 제안합니다.


In [29]:
translator('hello, my name is woojin choi. my hobby is play the FC-online.')

[{'translation_text': '안녕하세요, 제 이름은 우진 최입니다. 제 취미는 FC 온라인 게임을 하는 것입니다.'}]

In [54]:
import re

_url_pattern = re.compile(r"(https?://\S+|www\.\S+)")
_img_ext_pattern = re.compile(r"\b\w+\.(jpg|jpeg|png|gif|bmp|webp)\b", re.I)

def clean_text(s):
    s = _url_pattern.sub("", s)                   # URL 제거
    s = _img_ext_pattern.sub("", s)               # 이미지 파일명 토큰 제거
    s = re.sub(r"\s+", " ", s).strip()            # 공백 정리
    s = s.replace("’","'").replace("“","\"").replace("”","\"")  # 따옴표 정규화
    return s

clean_text(mbti_df['text'][0]).split('|||')

['@Pericles216 @HierBeforeTheAC @Sachinettiyil The Pope is infallible, this is a catholic dogma It doesn\'t mean the… @Pericles216 @Sachinettiyil And by perpetually it entails that the church herself can elect new po… @Pericles216 @Sachinettiyil You open the door to unironic nonsense when you believe this nonsense.… @Pericles216 @Sachinettiyil You don\'t know your own faith lol you just tried to say that "perpetua… @Pericles216 @Sachinettiyil That\'s like me saying that I\'m gonna give you a bike that will never b… @Pericles216 @Sachinettiyil You have a dumb view of God\'s preservation of the church. God doesn\'t… @Pericles216 @Sachinettiyil It\'s not heretical, all sedes do is take 2 quotes out of context from… @Pericles216 @Sachinettiyil Vatican 2 is not a new religion you moron lol it\'s an ecumenical counc… @Pericles216 @Sachinettiyil The issue isn\'t the mere vacancy it\'s the fact you don\'t think it\'s po… @Pericles216 @Sachinettiyil Denying dogmas and then saying "muh mystery" 

In [56]:
translator(clean_text(mbti_df['text'][0]).split('|||'))

[{'translation_text': '@Pericles216 @Sachinettiyil @HierBeforeTheAC @Sachinettiyil The Pope is infallible You don\'t know your own faith lol You just tried to say that "perpetua... @Pericles216 @Sachinettiyil It doesn\'t mean the... @Pericles216 @Sachinettiyil And by perpetually it entails that the church itself can elect new po... @Pericles216 @Sachinettiyil @Sachinettiyil @Pericles216 @Sachinettiyil @Pericles216 @Sachinettiyil @Pericles2 @Sachinettiyil @Sachinettiyil @Pericles2 @Sachinettiyil @Sachinettiyil @Sachinettiyil @Sachinettiyil @Sachinettiyil @Pericyl @Sachinettiyil @Pericyl @Sachinettiyil @Pericyl @Sachinettiyil @Pericyl @Sachinettiyil @Pericyl @Sachinettiyil @Pericyl @Sachinettiyil @Pericyl @Sachinettiyil @Pericyl @Pericyl @Sachinettiyil @Pericyl @Sachinettiyil @Sachintillan @Sachintillan @Pericyl @Pericyl @Sachintillan @Pericyllan @Sachintillan @Pericyllan @Sachintillan @Sachin Pericyllan @Pericyllan @SachintillanStillanStillanStillanStillanStillanStillanStillanStillanSti

In [3]:
from collections import Counter

words = list(mbti_df["text"].apply(lambda x: x.split()))
words = [x for y in words for x in y]
Counter(words).most_common(40)

NameError: name 'mbti_df' is not defined